In [1]:
import django, sys, os
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from itertools import product, combinations
from utils.text import *
from pycountry_convert import  country_alpha2_to_continent_code, country_alpha3_to_country_alpha2

import pandas as pd

In [2]:
df = pd.read_csv('IdLabelPrediction.csv')
df.head()

,id,label,prediction
0,305670,NaN,0.020998
1,3639141,NaN,0.040880
2,3390880,NaN,0.018189
3,3390419,NaN,0.232776
4,3679626,NaN,0.009844


In [3]:
included = df[(df['prediction']>0.35) | (df['label']==1)]
len(included.id.unique())

10633

In [4]:
df = pd.DataFrame.from_dict(
    Doc.objects.filter(id__in=included.id).values('id','title','content','PY','wc__oecd')
)


In [5]:
print(df.shape)
df.to_csv('../DIFID/data/doc_information.csv',index=False)
df.head()

(11193, 5)


,id,title,content,PY,wc__oecd
0,342890,Regional Warming and Emerging Vector-Borne Zoo...,We analyze through a climatic model the influe...,2014.0,Engineering and Technology
1,588520,Organic production systems: Sustainability ass...,Even though organic practices are getting more...,2016.0,Agricultural Sciences
2,115168,Air pollutant emissions inventory of large-sca...,For climate protection and energy security rea...,2016.0,Engineering and Technology
3,135179,Epidemiological analysis of malaria and its re...,"This work comprises a spatial, temporal and st...",2013.0,Natural Sciences
4,608857,Changes in volcanic hazard exposure in the Nor...,This investigation frames volcano disaster pot...,2015.0,Natural Sciences


In [9]:
from mordecai import Geoparser
geo = Geoparser()

geo.geoparse("I took the tube from Oxford Circus to London Bridge, via Bank")

[{'country_conf': 0.611725,
  'country_predicted': 'GBR',
  'geo': {'admin1': 'England',
   'country_code3': 'GBR',
   'feature_class': 'S',
   'feature_code': 'BDG',
   'geonameid': '6619889',
   'lat': '51.50821',
   'lon': '-0.08763',
   'place_name': 'London Bridge'},
  'spans': [{'end': 51, 'start': 38}],
  'word': 'London Bridge'}]

In [10]:
places = []
geos = []

for i, row in df.iterrows():
    
    t = row['title'] + row['content']
    t = t.split("Copyright (C)")[0] 
    t = re.split("\([C-c]\) [1-2][0-9]{3} Elsevier",t)[0] 
    t = t.split("Published by Elsevier")[0] 
    t = t.split("Copyright. (C)")[0] 
    t = re.split("\. \(C\) [1-2][0-9]{3} ",t)[0] 
    t = re.split("\. \(C\) Copyright",t)[0]     
    
    gp = geo.geoparse(t)
    
    rplaces = []
    continent = None
    for p in gp:
        try:
            a2 = country_alpha3_to_country_alpha2(p["country_predicted"])
            continent = country_alpha2_to_continent_code(a2)
        except:
            pass
        if "geo" in p:
            try:
                a2 = country_alpha3_to_country_alpha2(p["geo"]["country_code3"])
                continent = country_alpha2_to_continent_code(a2)
            except:
                pass
            p["geo"]["doc_id"] = row['id']
            p["geo"]["word"] = p["word"]
            p["geo"]["country_predicted"] = p["country_predicted"]
            p["geo"]["country_conf"] = p["country_conf"]
            geos.append(p["geo"])
        rplaces.append(p)
    df.loc[i,"continent"] = continent
    df.loc[i,"places"] = len(rplaces)

In [12]:
geo_df = pd.DataFrame.from_dict(geos)
geo_df.to_csv('data/geo_df.csv', index=False)
geo_df.head()

,admin1,lat,lon,country_code3,geonameid,place_name,feature_class,feature_code,doc_id,word,country_predicted,country_conf
0,Shandong,36.66833,116.99722,CHN,1805753,Jinan,P,PPLA,1061,Jinan,CHN,0.999811
1,Shandong,36.66833,116.99722,CHN,1805753,Jinan,P,PPLA,1061,Jinan,CHN,0.999811
2,NA,35,105,CHN,1814991,People’s Republic of China,A,PCLI,1061,China,CHN,0.999764
3,New South Wales,-33.78771,151.2838,AUS,7289168,Northern Beaches,L,AREA,1672,Northern,AUS,0.833024
4,New South Wales,-33.78771,151.2838,AUS,7289168,Northern Beaches,L,AREA,1672,Northern,AUS,0.833024


In [10]:
geo_df.head(100)

,admin1,lat,lon,country_code3,geonameid,place_name,feature_class,feature_code,doc_id,word,country_predicted,country_conf
0,Aquitaine-Limousin-Poitou-Charentes,44.69935,1.24017,FRA,3018220,Florimont-Gaumier,P,PPL,1305496,GGME,FRA,0.731971
1,Aquitaine-Limousin-Poitou-Charentes,44.69935,1.24017,FRA,3018220,Florimont-Gaumier,P,PPL,1305496,GGME,FRA,0.731971
2,Oklahoma,35.44951,-97.3967,USA,4542765,Midwest City,P,PPL,615984,Midwest,USA,0.999811
3,NA,39.76,-98.5,USA,6252001,United States,A,PCLI,615984,United States,USA,0.999811
4,West Virginia,39.23942,-80.48367,USA,4822880,Southwest District,A,ADMD,615984,Southwest,USA,0.939277
...,...,...,...,...,...,...,...,...,...,...,...,...
95,NA,15.5,-90.25,GTM,3595528,Republic of Guatemala,A,PCLI,1296156,Guatemala,GTM,0.999931
96,Petén,16.99571,-89.81366,GTM,3591406,Lake Petén Itzá,H,LK,1296156,Lake Peten Itza,GTM,0.999931
97,NA,15.5,-90.25,GTM,3595528,Republic of Guatemala,A,PCLI,1296156,Guatemala,GTM,0.999931
98,Petén,16.99571,-89.81366,GTM,3591406,Lake Petén Itzá,H,LK,1296156,Peten Itza,GTM,0.999931
